# 문서 유사도 측정
 - 클래스 초기화: 문장 리스트
 - 출력1: 벡터별 TD-IDF 출력
 - 출력2: 벡터간 코사인 유사도 계산 및 출력 (벡터 목록 중 2개만 추출하여 계산)

In [155]:
import nltk
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class TextSimilarityExample:
    # __init__ ->
    def __init__(self):
        self.statements = [
            'ruled india',
            'Chalukyas ruled Badami',
            'So many kingdoms ruled India',
            'Lalbagh is a botanical garden in India'
        ]
        
# 배운 걸로 재 구현 해보자
    def TF(self, sentence):
        # .lower를 사용해서 전부 소문자로 변형 후 token화
        # 근데 왜 소문자로 만들어야 하나요?
        words = nltk.word_tokenize(sentence.lower()) 

        # 단어 빈도수 count
        count_dict = {} # 숫자를 셀 빈 dictionary를 만든다. 
        for word in words: # words 리스트 안에 word를 하나씩 꺼내고
            count_dict.setdefault(word, 0) # 기본 값은 0으로 둔다. 
            count_dict[word] += 1 # 같은 이름을 가진 word가 등장하면 1씩 += 해준다. 
        
        # TF 구하기
        dictionary = {} # TF를 구하는 새로운 dictionary
        for key in count_dict.keys(): # 딕셔너리의 key값, 즉, 단어들을 하나씩 가져온다.
            norm = count_dict[key]/float(len(words)) # val 값을 전체 단어 수로 나누어준다. 빈도수 측정 공식
            dictionary[key] = norm # 값을 빈도값으로 재정의 한다.
        return dictionary # 딕셔너리를 반환한다.

    def IDF(self):
        # 먼저 idf를 구할 공식을 def idf 로 정의한다.
        def idf(TotalNumberOfDocuments, NumberOfDocumentsWithThisWord):
            return 1.0 + math.log(TotalNumberOfDocuments/NumberOfDocumentsWithThisWord)
        
        numDocuments = len(self.statements) # 전체 문장의 개수 (이걸 왜 Documents라고 하지?)
        uniqueWords = {}
        idfValues = {}
        
        for sentence in self.statements: # 문장을 문장리스트에서 하나씩 가져온다.
            for word in nltk.word_tokenize(sentence.lower()): # token화 시킨다.
                if word not in uniqueWords: # 만약 단어가 빈 딕셔너리에 없으면, 
                    uniqueWords[word] = 1 # 그 단어를 추가하고 1값을 준다.
                else:
                    uniqueWords[word] += 1 # 만약 단어가 있으면, += 1을 해준다.
                    
        for word in uniqueWords: # 다시 단어를 꺼내온다.
            idfValues[word] = idf(numDocuments, uniqueWords[word]) # 공식을 통해 idf 값을 구한다.
        return idfValues

    # TF-IDF 구하는 공식
    def TF_IDF(self, query): 
        words = nltk.word_tokenize(query.lower()) # 먼저 단어들을 토큰화
        idf = self.IDF() # idf는 위에서 정의한 IDF를 그대로 활용한다.
        vectors = {} # vector 값을 받을 빈 딕셔너리 생성
        for sentence in self.statements: # 문장리스트에서 문장을 꺼내온다.
            tf = self.TF(sentence) # TF 역시 위에서 정의한 TF를 그대로 활용한다. 
            
            for word in words: # 단어를 하나씩 꺼내온다.
                tfv = tf[word] if word in tf else 0.0 # tfv의 값은 단어가 tf에 있을 때만 받고 ,그 외는 0
                idfv = idf[word] if word in idf else 0.0 # idfv의 값은 단어가 idf에 있을 때만 받고, 그 외는 0
                mul = tfv * idfv # 곱해준다. # 이거 왜 곱하는거야?!?!
                
                if word not in vectors: # 그 다음, 단어가 vector에 없으면 (빈 딕셔너리기 때문에 당연히 없음)
                    vectors[word] = [] # 벡터의 단어를 key로 하는 key와 빈 리스트 val 값을 만들고,
                vectors[word].append(mul) # 거기에 mul 값을 추가한다.
        print("Vectors: ", vectors)
        return vectors # 벡터 완성

    def displayVectors(self, vectors):
        print(self.statements) # 문장들 확인
        for word in vectors: # 만약 단어가 벡터안에 있으면, 
            print("{} -> {}".format(word, vectors[word])) # 단어는 -> 이 벡터로 변했다.

####################################### 여기까지 이해 끝냄 #######################################            
    
# 아나
    def cosineSimilarity(self):
        vec = TfidfVectorizer() # vec는 TF-IDF Vectorizer를 활용
        matrix = vec.fit_transform(self.statements) # 문장을 바로 박아서 matrix를 생성한다.

        # 1이상 5미만의 for문을 돌린다. 
        for j in range(1, 5):
            i = j - 1
            # j = 1, 2, 3, 4
            # i = 0, 1, 2, 3
            print("\tsimilarity of document {} with others".format(i)) # 문장 위치(인덱스)를 나타내는 문장 출력
            similarity = cosine_similarity(matrix[i:j], matrix) # 왜 0과 1의 유사도지? 0과 0이여야지 않나?
            print(similarity)

    def demo(self):
        inputQuery = self.statements[0]
        vectors = self.TF_IDF(inputQuery)
        
        inputQuery2 = self.statements[1]
        vectors2 = self.TF_IDF(inputQuery2)
        
        self.displayVectors(vectors)
        self.cosineSimilarity()

similarity = TextSimilarityExample()
similarity.demo()

Vectors:  {'ruled': [0.6438410362258904, 0.42922735748392693, 0.2575364144903562, 0.0], 'india': [0.6438410362258904, 0.0, 0.2575364144903562, 0.18395458177882582]}
Vectors:  {'chalukyas': [0.0, 0.7954314537066303, 0.0, 0.0], 'ruled': [0.6438410362258904, 0.42922735748392693, 0.2575364144903562, 0.0], 'badami': [0.0, 0.7954314537066303, 0.0, 0.0]}
['ruled india', 'Chalukyas ruled Badami', 'So many kingdoms ruled India', 'Lalbagh is a botanical garden in India']
ruled -> [0.6438410362258904, 0.42922735748392693, 0.2575364144903562, 0.0]
india -> [0.6438410362258904, 0.0, 0.2575364144903562, 0.18395458177882582]
	similarity of document 0 with others
[[1.         0.29088811 0.46216171 0.19409143]]
	similarity of document 1 with others
[[0.29088811 1.         0.13443735 0.        ]]
	similarity of document 2 with others
[[0.46216171 0.13443735 1.         0.08970163]]
	similarity of document 3 with others
[[0.19409143 0.         0.08970163 1.        ]]


## TEST

In [133]:
sentence = "djieljfAAAEJF"
sentence.lower()

'djieljfaaaejf'

In [134]:
nltk.FreqDist?

In [135]:
words = ['사과','바나나','딸기']
a = nltk.FreqDist(words)
a

FreqDist({'사과': 1, '바나나': 1, '딸기': 1})

In [136]:
dict.keys?

In [129]:
vectors = {'ruled': [0.6438410362258904, 0.42922735748392693, 0.2575364144903562, 0.0], 'india': [0.6438410362258904, 0.0, 0.2575364144903562, 0.18395458177882582]}

# 실패한 코드 
# 코사인 유사도 구하기
def cosineSimilarity(self):
    temp_list = []
    # x와 y, 두 벡터의 코사인 유사도를 계산하는 함수

    # for문돌려서 벡터 두개 꺼내오고,
    for word in vectors:
        # print(vectors[word])
        temp_list.append(vectors[word])

    print(temp_list)
    # x, y로 수정한다음에
    x = temp_list[0]
    y = temp_list[1]
    
    nominator = np.dot(x,y) # 분자
    donominator = np.linalg.norm(x) * np.linalg.norm(y) # 분모
    similarity = nominator / donominator
    print(similarity)

## 네이버 뉴스 요약
 - 입력: 네이버 뉴스, url, 요약 비율
 - 출력: matrix 혹은 그래프 활용 textrank 구현 이용한 문서 요약
 
출력: 요약문(요약비율 적용), 원문

In [137]:
import requests
from bs4 import BeautifulSoup

def summarizeNaverNews(url, ratio):
    

In [196]:
import requests
from bs4 import BeautifulSoup
from gensim.summarization.summarizer import summarize
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import networkx as nx
import math
import matplotlib.pyplot as plt
import re

url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=018&aid=0004435771'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
# soup
content = soup.find('div', id='articleBodyContents').text
# 나중에 개행문자와 '오류를 우회하기 위한 함수 추가와 같은 것들 제거할 것'
Text = content
Text = Text.strip()
Text = Text.lstrip()
Text = Text.replace('\n','')
Text = Text.replace('// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}','')
Text = Text.replace("이윤화 (akfdl34@edaily.co.kr)네이버 홈에서 ‘이데일리’ 뉴스 [구독하기▶]꿀잼가득 [영상보기▶] , 청춘뉘우스~ [스냅타임▶]＜ⓒ종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지＞",'')
# print(Text)


# 문장간 유사도 측정 (BOW를 활용한 코사인 유사도 측정)
def sentence_similarity(sentence1, sentence2):
    # 각 문장을 소문자로 변환
    sentence1 = [word.lower() for word in sentence1.split()]
    sentence2 = [word.lower() for word in sentence2.split()]
    
    # BOW 생성을 위한 unique한 단어로 배열 생성
    words_ls = list(set(sentence1 + sentence2))
    
    bow1 = [0] * len(words_ls)
    bow2 = [0] * len(words_ls)
    
    # 첫번째 문장 BoW 생성
    for word in sentence1:
        bow1[words_ls.index(word)] += 1
        
    # 두번째 문장 BoW 생성
    for word in sentence2:
        bow2[words_ls.index(word)] += 1
        
    return cosine_similarity(bow1, bow2)

# 코사인 유사도 (1. 단어의 표현 예제 참고)
def cosine_similarity(x, y):
    # x와 y, 두 벡터의 코사인 유사도를 계산하는 함수
    nominator = np.dot(x,y) # 분자
    donominator = np.linalg.norm(x)*np.linalg.norm(y) # 분모
    return nominator / donominator

##############################################

def sentences(text):
#     print(text.split('.'))
    return sent_tokenize(text)

def connect(nodes):
    return [(start, end, sentence_similarity(start, end)) for start in nodes for end in nodes if start is not end]

def rank(nodes,edges):
    graph=nx.diamond_graph()
    graph.add_nodes_from(nodes)
    graph.add_weighted_edges_from(edges)
    
#     nx.draw(graph)
#     plt.show()
    
    return nx.pagerank(graph)

def summarize(text,num_summaries=6):
    nodes=sentences(text)
    edges=connect(nodes)
    scores=rank(nodes,edges)
    #print(nodes)
    return sorted(scores,key=scores.get)[:num_summaries]

summary = summarize(Text, 3)
for sentence in summary:
    print('='*15)
    print(sentence)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


8월부터 이마트, 홈플러스 등 전국 대형매장 판매온라인, 한정판매 완판 기록하며 소비자 인기 입증포기하지 마라탕면.
한정판 세트를 단독 판매한 11번가에서는 판매 시간 동안 실시간 쇼핑 검색어 1위를 달리며 높은 관심을 입증했고, 전체 판매량 3위에 이름을 올렸다.풀무원은 앞으로 11번가 외에도 다양한 온라인 채널을 통해 소비자를 위한 세트 구성 및 프로모션을 진행할 계획이다.이기욱 풀무원식품 생면식감 사업부 PM(Product Manager)은 “’포기하지 마라탕면’은 두 차례의 온라인 한정 판매서 단기간에 완판되는 저력을 보여줬다.
더불어, 라면 조리 시 기름으로 인해 맛이 퍼지는 현상이 나타나지 않아 정통 마라탕에 가까운 국물 맛을 구현해냈다.


In [144]:
# def summarize(text, linesinSummary=10):
#     text = sent_tokenize(text)
#     weighted_edge = buildMatrix(text)
#     score = scoring(weighted_edge)
#     # print(score)
    
#     # -item[1] 은 오름차순 정렬
#     # item[1] 은 내림차순 정렬
#     rankedSentenceIndexes = [item[0] for item in sorted(enumerate(score), key=lambda item: -item[1])]
#     selectedSentences = sorted(rankedSentenceIndexes[:linesinSummary])
#     summary = itemgetter(*selectedSentences)(text)
    
#     return summary